In [1]:
# Comment the following cell if running through the command :
# papermill runnable_HPO.ipynb executed_HPO.ipynb -f params_for_HPO.yaml --kernel jupyter.jax_env

In [2]:

global_seed = 88
# Parameters specfific to the benchmark
benchmark_settings = {"benchmark_name" : "pMNIST",
                      "difficulty" : "standard",
                      "num_tasks" : 4,
                      "train_percentage" : 0.2,
                      "num_val_benchmarks" : 5,
                      "batch_size" : 128}

# Parameters specific to the method
method_settings = {"method_name" : "GroHess",
                   "grow_from" : "output",
                   "hessian_percentile" : 95,
                   "grad_percentile" : 95,
                   "num_inputs" : 28*28,
                   "num_hidden_root" : 100,
                   "num_outputs" : 10,
                   "loss_name" : "CE",
                   "optimizer_name" : "Adam"}

# Parameters specific to HPO
HPO_settings = {"HPO_name" : "greedy_HPO",
                "n_trials" : 3,
                "lr" : (5e-5, 2e-3),
                "num_epochs" : (2, 6)}

# Parameters specific to the visualization
visualization_settings = {"visualize_accs_matrix" : True,
                          "visualize_avg_acc_curve" : True,
                          "visualize_best_params" : True,}

In [3]:
import sys
import os

sys.path.append("Methods/" + method_settings["method_name"])
sys.path.append("HPO_lib/benchmark_loaders")

path = os.path.dirname(os.path.abspath("__file__"))
data_path = path + "/data"

## Imports

In [4]:
from lib.abstract_torch import get_device
from lib.dataloader import get_task_loaders
from lib.models import ANN
from lib.train import train
from lib.test import test
from HPO_lib.visualize import visualize_accs_matrix, visualize_avg_acc_curve, visualize_best_params, visualize_val_accs_matrix, visualize_accuracy_through_benchmarks, visualize_violin
from HPO_lib.get_benchmarks import get_benchmarks
from HPO_lib.run_HPO import run_HPO

import numpy as np
import copy
import optuna
from functools import partial

import warnings
warnings.filterwarnings('ignore')

/home/mathis/miniconda3/envs/.jax_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = get_device(1)

## Get benchmarks

In [6]:
benchmarks_list = get_benchmarks(benchmark_settings, global_seed)

## Run HPO

In [7]:
test_accs_matrix, best_params_list, model = run_HPO(HPO_settings, method_settings, benchmarks_list[0], device, global_seed)

[I 2024-07-04 16:50:51,489] A new study created in memory with name: Search number 1



--------------------------------------------------
LEARNING TASK 1


100%|██████████| 4/4 [00:05<00:00,  1.36s/it]
[I 2024-07-04 16:51:01,776] Trial 0 finished with value: 91.75594666666667 and parameters: {'lr': 0.0013127245462383958, 'num_epochs': 4}. Best is trial 0 with value: 91.75594666666667.
100%|██████████| 6/6 [00:08<00:00,  1.33s/it]
[I 2024-07-04 16:51:14,612] Trial 1 finished with value: 92.58317333333342 and parameters: {'lr': 0.001080265691296867, 'num_epochs': 6}. Best is trial 1 with value: 92.58317333333342.
100%|██████████| 5/5 [00:06<00:00,  1.32s/it]
[I 2024-07-04 16:51:26,072] Trial 2 finished with value: 92.74338666666672 and parameters: {'lr': 0.0014149828218016016, 'num_epochs': 5}. Best is trial 2 with value: 92.74338666666672.
  0%|          | 0/5 [00:00<?, ?it/s]

In and out sizes :
fc1 : in = 784, out = 100
fc2 : in = 100, out = 100
fc3 : in = 100, out = 10


 20%|██        | 1/5 [00:01<00:05,  1.33s/it]

...

 67%|██████▋   | 4/6 [00:05<00:02,  1.28s/it]

In and out sizes :
fc1 : in = 784, out = 122
fc2 : in = 122, out = 100
fc3 : in = 100, out = 10


 83%|████████▎ | 5/6 [00:06<00:01,  1.28s/it]

In and out sizes :
fc1 : in = 784, out = 122
fc2 : in = 122, out = 100
fc3 : in = 100, out = 10


100%|██████████| 6/6 [00:07<00:00,  1.28s/it]
